## Data Wrangling

source of data: https://nces.ed.gov/EDAT/Data/Zip/HSLS_2016_v1_0_CSV_Datasets.zip

source of layout data: https://nces.ed.gov/EDAT/Data/Zip/HSLS_2016_v1_0_CodeBook_Layout.zip

In [2]:
import pandas as pd
import numpy as np

The csv file is quite large, but not so large that it can't fit in memory, however, it fits in much more easily if we take away some of the columns that won't be useful to the analysis right away. These include columns that are suppressed for public use due to privacy concerns, columns that include statistical weights used to create accurate aggregates of the data, and columns indicating where values have been imputed.

Though these other columns, particularly the weights, will be used later, here we initially only extract the data columns so that we can read in a table in a reasonable amount of time.

In [7]:
#create variables for file locations
filepath = '/Users/chrismalec/Documents/Springboard/CapstoneProjectOne/'
studentdatafile = 'HSLS_2016_v1_0_CSV_Datasets/hsls_16_student_v1_0.csv'
studentcodebookfile = 'HSLS_2016_v1_0_CodeBook_Layout/Layout_STUDENT.txt'
schooldatafile = 'HSLS_2016_v1_0_CSV_Datasets/hsls_09_school_v1_0.csv'
#import a small sample of the file
with open(filepath+studentdatafile) as file:
    student_df = pd.read_csv(file,nrows = 10)
    file.close()

#Remove columns with '-5' and create list of column names removed.
#Create separate list of columns for weighting and imputation.
#The remaining columns will constitute the database.
student_suppressed_columns = []
student_weight_columns = []
student_imputed_columns = []
student_columns = []
for column in student_df.columns:
    if (student_df[column] == -5).all():
        student_suppressed_columns.append(column)
    elif column[0] == 'W':
        student_weight_columns.append(column)
    elif column[-2:] == 'IM':
        student_imputed_columns.append(column)
    else:
        student_columns.append(column)

In [8]:
#Import the selected columns from the file for all rows.
with open(filepath+studentdatafile) as file:
    student_df = pd.read_csv(file,usecols = student_columns)
    file.close()

There are several variables that indicate the students' dropout status and whether or not they have ever dropped out. This variable was chosen as the label because it indicated the outcome of the student as stated on their transcript four years after the ninth grade study. Other columns may be removed later due to high (trivial) correlation with the label.

In [9]:
#Create series of labels and remove it from the dataframe.
labels = student_df.pop('X3TOUTCOME')

As the project moves forward, it will be convenient to easily move between the numbers that denote various responses to the surveys and the human readable responses that the number represent. For example, the column 'S1ABILITYBA' represents a response to whether the '9th grader thinks he/she has the ability to complete a Bachelor's degree.' The possible answers are coded in the data table as 1,2,3,4,-8, and -9 which correspond to "Definitely not", "Probably not", "Probably", "Definitely", "Unit non-response", and "Missing."

I create a dictionary from the documentation so that I can type student_value_labels['S1ABILITYBA']['4'] and obtain "Definitely" for use as a label in a graph or figure. Similarly, I include the description of the variable under the key 'desc'.

In [10]:
#create dictionary from the student codebook
#Start creating dictionary.
student_value_labels = {}
coded_columns = student_columns + student_imputed_columns + [labels.name]
with open(filepath+studentcodebookfile,'r') as file:
    #Reads file until it reaches the descriptions
    for line in file:
        if '/* Variable Names, Locations, and Descriptions */' in line:
            break
    
    #Takes first ASCII column and associates it with a data column
    for line in file:
        if len(line.split()) > 0:
            new_key = line.split()[0].strip()
        
        #places description in dictionary under 'desc'
        #reads until the value labels are reached
        if new_key in coded_columns:
            student_value_labels[new_key]={}
            desc = ' '.join(line.split()[2:])
            student_value_labels[new_key]['desc'] = desc
        elif '/* Variable Value Labels */' in line:
            break
        else:
            continue
    
    #key places additional key value pairs to specify the response associated with the number in the data table.       
    for line in file:
        if line.strip() in student_value_labels.keys():
            key = line.strip()
        #this comes up for some reason, not sure why there is a column or two that has no description but has a value mapping.
        elif line.strip() in coded_columns:
            key = line.strip()
            student_value_labels[key]={}
        elif line.strip()[:2] == '-5':
            continue
        elif line.strip() in student_suppressed_columns:
            continue
        elif line.strip() in student_weight_columns:
            continue
        #unambiguous end of file
        elif line == '':
            break
        else:
            key_value = line.split('=')
            student_value_labels[key][key_value[0].strip()] = key_value[1].strip()
file.close()

All negative numbers are missing data. The different values denote reasons for that the data is missing, which I can load from the original data if it turns out to be useful. I replace them with an nan value to assist in future data operations.

In [11]:
#replace negative numbers with nan
student_df = student_df.replace([x for x in range(-9,0)],np.nan)

print('There are '+str(student_df.isna().values.sum())+' out of '+str(student_df.size)+' missing values in the dataframe.')
print('The dataframe contains '+str(student_df.shape[1])+' features and '+str(student_df.shape[0])+' observations.')

There are 24183783 out of 63904657 missing values in the dataframe.
The dataframe contains 2719 features and 23503 observations.


The data in the table is from several sources, and four different collection times. The first two letters in each column name specify the source of the data and the collection time of the data.  So as not to mix time periods, it may be useful to split the dataframe into four dataframes that contain data from the four collection times. After all, I don't want to imply that something that happened in a student's post-secondary career affected their decision to drop out in high school.

First character code:
Composite variables = X
Student = S
Parent = P
Mathematics teacher = M
Science teacher = N
Administrator = A
Counselor = C
Weights = W

Second character code:
Base Year = 1
First Followup = 2
Second Followup = 3
Third Followup = 4

In [33]:
#create logical masks to separate tables based on time
mask_BY = [(x[1]=='1')| (x=='STU_ID') for x in student_df.columns]
mask_F1 = [x[1]=='2' for x in student_df.columns]
mask_F2 = [x[1]=='3' for x in student_df.columns]
mask_F3 = [x[1]=='4' for x in student_df.columns]

#slice original table into four separate tables.
df_BY = student_df.loc[:,mask_BY]
df_F1 = student_df.loc[:,mask_F1]
df_F2 = student_df.loc[:,mask_F2]
df_F3 = student_df.loc[:,mask_F3]

In [13]:
#Store pickled versions of the dataframes locally so I don't have to redo this every time.
df_BY.to_pickle(filepath+'df_BY')
df_F1.to_pickle(filepath+'df_F1')
df_F2.to_pickle(filepath+'df_F2')
df_F3.to_pickle(filepath+'df_F3')

In [14]:
#Import the selected columns from the file for all rows.
with open(filepath+studentdatafile) as file:
    weights_df = pd.read_csv(file,usecols = student_weight_columns)
    file.close()

In [62]:
with open(filepath+schooldatafile) as file:
    school_df = pd.read_csv(file)
    file.close()
school_columns = school_df.columns

In [41]:
school_student_columns = []
for column in school_columns:
    if column in student_columns:
        school_student_columns.append(column)

In [64]:
sch_stu_df = df_BY.loc[:,['STU_ID']+school_student_columns].groupby(['X1AQDATE','X1CQDATE']).count()
school_df

,Sch_ID,X1NCESID,W1SCHOOL,STRAT_ID,PSU,X1CONTROL,X1LOCALE,X1REGION,X1CENDIV,X1STATESAMPL,...,W1SCHOOL191,W1SCHOOL192,W1SCHOOL193,W1SCHOOL194,W1SCHOOL195,W1SCHOOL196,W1SCHOOL197,W1SCHOOL198,W1SCHOOL199,W1SCHOOL200
0,1001,-5,2.331907,-5,-5,1,1,3,-5,-5,...,0.000000,0.000000,3.601229,0.000000,5.203120,0.000000,0.000000,0.000000,0.000000,0.000000
1,1002,-5,4.615802,-5,-5,1,2,3,-5,-5,...,0.000000,5.117174,5.931073,0.000000,0.000000,0.000000,5.537744,6.027139,0.000000,0.000000
2,1003,-5,8.810384,-5,-5,1,1,3,-5,-5,...,0.000000,17.768776,13.514905,17.794190,20.618902,16.673944,24.577378,0.000000,0.000000,0.000000
3,1004,-5,8.845971,-5,-5,2,2,2,-5,-5,...,0.000000,9.884404,0.000000,0.000000,0.000000,6.866897,25.398480,6.175511,21.028035,5.663044
4,1005,-5,10.807970,-5,-5,1,2,3,-5,-5,...,21.067261,12.796792,11.042852,0.000000,22.656201,16.703235,0.000000,20.871432,22.551557,0.000000
5,1006,-5,10.023618,-5,-5,1,2,3,-5,-5,...,16.106712,0.000000,37.070575,12.291824,0.000000,24.603154,0.000000,28.206375,0.000000,11.276811
6,1007,-5,5.390407,-5,-5,1,1,3,-5,-5,...,0.000000,0.000000,0.000000,0.000000,8.821863,9.634202,0.000000,0.000000,0.000000,10.009339
7,1008,-5,7.657878,-5,-5,1,1,4,-5,-5,...,0.000000,0.000000,0.000000,66.149633,35.731591,30.202224,0.000000,33.611170,0.000000,45.937731
8,1009,-5,21.281653,-5,-5,2,3,3,-5,-5,...,0.000000,0.000000,0.000000,0.000000,25.900863,0.000000,0.000000,16.158162,0.000000,0.000000
9,1010,-5,6.015197,-5,-5,1,3,3,-5,-5,...,0.000000,40.213143,0.000000,0.000000,0.000000,0.000000,39.618218,32.134719,0.000000,28.045794


In [66]:
student_suppressed_columns

['SCH_ID',
 'X1NCESID',
 'X2NCESID',
 'STRAT_ID',
 'PSU',
 'X1ASIAN',
 'X1PACISLE',
 'X1AMINDIAN',
 'X1HISPTYPE',
 'X1ASIANTYPE',
 'X1NATIVELANG',
 'X1PAR1OCC6',
 'X1PAR1OCC_STEM2',
 'X1PAR2OCC6',
 'X1PAR2OCC_STEM2',
 'X1MOMOCC6',
 'X1MOMOCC_STEM2',
 'X1DADOCC6',
 'X1DADOCC_STEM2',
 'X1STU30OCC6',
 'X1STU30OCC_STEM2',
 'X1STUPRVSCHL_R',
 'X1SQINCAPABL',
 'X1CENDIV',
 'X1STATESAMPL',
 'X1STATE',
 'X1GRADESPAN',
 'X1FREELUNCH',
 'X1REPEAT9TH',
 'X1SCHAMIND',
 'X1SCHASIAN',
 'X1SCHBLACK',
 'X1SCHHISP',
 'X1SCHWHITE',
 'X2ASIAN',
 'X2PACISLE',
 'X2AMINDIAN',
 'X2HISPTYPE',
 'X2ASIANTYPE',
 'X2NATIVELANG',
 'X2PARRESP',
 'X2PAR1OCC6',
 'X2PAR1OCC_STEM2',
 'X2PAR2OCC6',
 'X2PAR2OCC_STEM2',
 'X2MOMOCC6',
 'X2MOMOCC_STEM2',
 'X2DADOCC6',
 'X2DADOCC_STEM2',
 'X2STU30OCC6',
 'X2STU30OCC_STEM2',
 'X2SQINCAPABL',
 'X2CENDIV',
 'X2STATE',
 'X2FREELUNCH',
 'X2AQSTAT',
 'X2AQDESIGNEE',
 'X2CQSTAT',
 'X3NCESID',
 'X3CENDIV',
 'X3STATE',
 'X3MATCHATMPT',
 'X3GEDSTATE',
 'X3HSCOMPDATE',
 'X3TLASTHS',
 '